In [73]:
import pandas as pd
import glob

In [74]:
results = list(glob.glob("../rsna24-data/models*/*/res*", recursive=True))
results[:2]

['../rsna24-data/models/tinynet_e.in1k-c3p1b16e20f14/result',
 '../rsna24-data/models/densenet121-c3p1b16e20f14/result']

In [75]:
import os
import re


def convert_to_float(s):
    try:
        return round(float(s), 3)
    except Exception as e:
        print(e)
        return 0.0

results_list = []
for result in results:
    result_dict = [os.path.basename(os.path.dirname(result))]
    # {
    #     "model": os.path.dirname(result),
    #     "data": []
    #     # "spinal_canal": [],
    #     # "subarticular": [],
    #     # "neural_foraminal": [],
    # }
    with open(result, "r") as f:
        txt = f.readlines()
        line_index = None
        for line in txt:
            if re.match("spinal_canal", line):
                line_index = "spinal_canal"
                result_dict.append(line_index)
            if re.match("subarticular", line):
                line_index = "subarticular"
                result_dict.append(line_index)
            if re.match("neural_foraminal", line):
                line_index = "neural_foraminal"
                result_dict.append(line_index)
            
            value_pattern = "(\\d+.\\d+)|nan"
            if line_index is not None and re.match("accuracy.*", line):
                result_dict.extend([convert_to_float(d) for d in re.findall(value_pattern, line)])
            if line_index is not None and re.match("normal.*", line):
                result_dict.extend([convert_to_float(d) for d in re.findall(value_pattern, line)])
            if line_index is not None and re.match("moderate.*", line):
                result_dict.extend([convert_to_float(d) for d in re.findall(value_pattern, line)])
            if line_index is not None and re.match("severe.*", line):
                result_dict.extend([convert_to_float(d) for d in re.findall(value_pattern, line)])
            
    results_list.append(result_dict)

len(results_list)

could not convert string to float: ''
could not convert string to float: ''
could not convert string to float: ''
could not convert string to float: ''
could not convert string to float: ''


25

In [76]:
results_df = pd.DataFrame(results_list, columns=[
    "model", "sp", "sp_acc", "spn_cw", "spn_a", "spm_cw", "spm_a", "sps_cw", "sps_a",
    "sb", "sb_acc", "sbn_cw", "sbn_a", "sbm_cw", "sbm_a", "sbs_cw", "sbs_a",
    "nf", "nf_acc", "nfn_cw", "nfn_a", "nfm_cw", "nfm_a", "nfs_cw", "nfs_a",
])
results_df.head()

,model,sp,sp_acc,spn_cw,spn_a,spm_cw,spm_a,sps_cw,sps_a,sb,...,sbs_cw,sbs_a,nf,nf_acc,nfn_cw,nfn_a,nfm_cw,nfm_a,nfs_cw,nfs_a
0,tinynet_e.in1k-c3p1b16e20f14,spinal_canal,0.730,11.333,0.787,0.333,0.170,0.248,0.638,subarticular,...,0.605,0.467,neural_foraminal,0.723,8.105,0.798,0.701,0.509,0.318,0.403
1,densenet121-c3p1b16e20f14,spinal_canal,0.712,20.682,0.752,0.367,0.208,0.238,0.766,subarticular,...,0.789,0.335,neural_foraminal,0.778,8.321,0.851,0.900,0.617,1.357,0.284
2,densenet169-c3p1b16e20f14,spinal_canal,0.804,9.421,0.888,1.500,0.057,0.342,0.574,subarticular,...,0.745,0.473,neural_foraminal,0.763,6.679,0.877,0.799,0.430,0.489,0.328
3,densenet161-c3p1b16e20f14,spinal_canal,0.760,9.463,0.845,0.143,0.038,0.228,0.489,subarticular,...,1.235,0.251,neural_foraminal,0.759,5.802,0.882,0.678,0.426,3.000,0.179
4,regnety_016.tv2_in1k-c3p1b16e20f14,spinal_canal,0.661,23.111,0.688,0.429,0.226,0.197,0.809,subarticular,...,0.829,0.347,neural_foraminal,0.779,6.050,0.908,0.902,0.397,0.667,0.299


# Spinal Canal

In [77]:
sp = results_df[["model", "sp", "sp_acc", "spn_cw", "spn_a", "spm_cw", "spm_a", "sps_cw", "sps_a"]]
sp.head()

,model,sp,sp_acc,spn_cw,spn_a,spm_cw,spm_a,sps_cw,sps_a
0,tinynet_e.in1k-c3p1b16e20f14,spinal_canal,0.730,11.333,0.787,0.333,0.170,0.248,0.638
1,densenet121-c3p1b16e20f14,spinal_canal,0.712,20.682,0.752,0.367,0.208,0.238,0.766
2,densenet169-c3p1b16e20f14,spinal_canal,0.804,9.421,0.888,1.500,0.057,0.342,0.574
3,densenet161-c3p1b16e20f14,spinal_canal,0.760,9.463,0.845,0.143,0.038,0.228,0.489
4,regnety_016.tv2_in1k-c3p1b16e20f14,spinal_canal,0.661,23.111,0.688,0.429,0.226,0.197,0.809


In [78]:
sp.sort_values("sp_acc", ascending=False).head(3)

,model,sp,sp_acc,spn_cw,spn_a,spm_cw,spm_a,sps_cw,sps_a
16,densenet201-DB-c3p1b16e20f14,spinal_canal,0.816,6.714,0.932,0.0,0.000,0.239,0.234
2,densenet169-c3p1b16e20f14,spinal_canal,0.804,9.421,0.888,1.5,0.057,0.342,0.574
15,densenet169-DB-c3p1b16e20f14,spinal_canal,0.782,8.108,0.871,0.0,0.000,0.270,0.511


In [79]:
sp.sort_values("spn_a", ascending=False).head(3)

,model,sp,sp_acc,spn_cw,spn_a,spm_cw,spm_a,sps_cw,sps_a
16,densenet201-DB-c3p1b16e20f14,spinal_canal,0.816,6.714,0.932,0.0,0.000,0.239,0.234
2,densenet169-c3p1b16e20f14,spinal_canal,0.804,9.421,0.888,1.5,0.057,0.342,0.574
15,densenet169-DB-c3p1b16e20f14,spinal_canal,0.782,8.108,0.871,0.0,0.000,0.270,0.511


In [80]:
sp.sort_values("spm_a", ascending=False).head(3)

,model,sp,sp_acc,spn_cw,spn_a,spm_cw,spm_a,sps_cw,sps_a
5,rexnet_150.nav_in1k-A-c9p1b16e20f14,spinal_canal,0.109,9.000,0.045,0.101,0.509,0.064,0.489
24,efficientnet_b0-DB-c3p1b16e20f14,spinal_canal,0.677,22.474,0.706,0.224,0.453,0.255,0.553
21,efficientnet_b4-DB-c3p1b16e20f14,spinal_canal,0.616,37.600,0.621,0.204,0.358,0.232,0.830


In [81]:
sp.sort_values("sps_a", ascending=False).head(3)

,model,sp,sp_acc,spn_cw,spn_a,spm_cw,spm_a,sps_cw,sps_a
17,efficientnet_b2-DB-c3p1b16e20f14,spinal_canal,0.499,49.667,0.493,0.158,0.170,0.155,0.957
18,efficientnet_b1-DB-c3p1b16e20f14,spinal_canal,0.682,23.000,0.722,0.500,0.019,0.212,0.915
19,xception41-DB-c3p1b16e20f14,spinal_canal,0.675,24.056,0.716,0.125,0.038,0.210,0.872


# Subarticual

In [82]:
sb = results_df[["model", "sb", "sb_acc", "sbn_cw", "sbn_a", "sbm_cw", "sbm_a", "sbs_cw", "sbs_a"]]
sb.head()

,model,sb,sb_acc,sbn_cw,sbn_a,sbm_cw,sbm_a,sbs_cw,sbs_a
0,tinynet_e.in1k-c3p1b16e20f14,subarticular,0.691,3.838,0.874,0.536,0.168,0.605,0.467
1,densenet121-c3p1b16e20f14,subarticular,0.702,3.295,0.906,0.630,0.190,0.789,0.335
2,densenet169-c3p1b16e20f14,subarticular,0.707,3.557,0.897,0.705,0.160,0.745,0.473
3,densenet161-c3p1b16e20f14,subarticular,0.713,3.289,0.921,0.663,0.243,1.235,0.251
4,regnety_016.tv2_in1k-c3p1b16e20f14,subarticular,0.699,3.420,0.902,0.505,0.183,0.829,0.347


In [83]:
sb.sort_values("sb_acc", ascending=False).head(3)

,model,sb,sb_acc,sbn_cw,sbn_a,sbm_cw,sbm_a,sbs_cw,sbs_a
3,densenet161-c3p1b16e20f14,subarticular,0.713,3.289,0.921,0.663,0.243,1.235,0.251
6,efficientnet_b2-c3p1b16e20f14,subarticular,0.713,3.767,0.912,0.673,0.254,0.731,0.293
2,densenet169-c3p1b16e20f14,subarticular,0.707,3.557,0.897,0.705,0.160,0.745,0.473


In [84]:
sb.sort_values("sbn_a", ascending=False).head(3)

,model,sb,sb_acc,sbn_cw,sbn_a,sbm_cw,sbm_a,sbs_cw,sbs_a
13,efficientnet_b4-c3p1b16e20f14,subarticular,0.706,3.320,0.926,0.533,0.119,0.723,0.359
3,densenet161-c3p1b16e20f14,subarticular,0.713,3.289,0.921,0.663,0.243,1.235,0.251
6,efficientnet_b2-c3p1b16e20f14,subarticular,0.713,3.767,0.912,0.673,0.254,0.731,0.293


In [85]:
sb.sort_values("sbm_a", ascending=False).head(3)

,model,sb,sb_acc,sbn_cw,sbn_a,sbm_cw,sbm_a,sbs_cw,sbs_a
21,efficientnet_b4-DB-c3p1b16e20f14,subarticular,0.570,6.940,0.598,0.376,0.534,0.538,0.461
20,efficientnet_b3-DB-c3p1b16e20f14,subarticular,0.519,8.274,0.526,0.348,0.515,0.368,0.485
24,efficientnet_b0-DB-c3p1b16e20f14,subarticular,0.639,5.815,0.739,0.436,0.429,0.537,0.389


In [86]:
sb.sort_values("sbs_a", ascending=False).head(3)

,model,sb,sb_acc,sbn_cw,sbn_a,sbm_cw,sbm_a,sbs_cw,sbs_a
15,densenet169-DB-c3p1b16e20f14,subarticular,0.598,4.799,0.684,0.341,0.340,0.528,0.509
16,densenet201-DB-c3p1b16e20f14,subarticular,0.578,5.990,0.633,0.345,0.425,0.519,0.503
14,densenet161-DB-c3p1b16e20f14,subarticular,0.603,5.310,0.686,0.356,0.369,0.526,0.491


# Nural Foraminal

In [87]:
nf = results_df[["model", "nf", "nf_acc", "nfn_cw", "nfn_a", "nfm_cw", "nfm_a", "nfs_cw", "nfs_a"]]
nf.head()

,model,nf,nf_acc,nfn_cw,nfn_a,nfm_cw,nfm_a,nfs_cw,nfs_a
0,tinynet_e.in1k-c3p1b16e20f14,neural_foraminal,0.723,8.105,0.798,0.701,0.509,0.318,0.403
1,densenet121-c3p1b16e20f14,neural_foraminal,0.778,8.321,0.851,0.900,0.617,1.357,0.284
2,densenet169-c3p1b16e20f14,neural_foraminal,0.763,6.679,0.877,0.799,0.430,0.489,0.328
3,densenet161-c3p1b16e20f14,neural_foraminal,0.759,5.802,0.882,0.678,0.426,3.000,0.179
4,regnety_016.tv2_in1k-c3p1b16e20f14,neural_foraminal,0.779,6.050,0.908,0.902,0.397,0.667,0.299


In [88]:
nf.sort_values("nf_acc", ascending=False).head(3)

,model,nf,nf_acc,nfn_cw,nfn_a,nfm_cw,nfm_a,nfs_cw,nfs_a
11,xception41-c3p1b16e20f14,neural_foraminal,0.798,6.703,0.912,1.085,0.462,1.136,0.373
9,densenet201-c3p1b16e20f14,neural_foraminal,0.797,5.519,0.937,1.126,0.386,1.800,0.269
4,regnety_016.tv2_in1k-c3p1b16e20f14,neural_foraminal,0.779,6.050,0.908,0.902,0.397,0.667,0.299


In [89]:
nf.sort_values("nfn_a", ascending=False).head(3)

,model,nf,nf_acc,nfn_cw,nfn_a,nfm_cw,nfm_a,nfs_cw,nfs_a
9,densenet201-c3p1b16e20f14,neural_foraminal,0.797,5.519,0.937,1.126,0.386,1.800,0.269
11,xception41-c3p1b16e20f14,neural_foraminal,0.798,6.703,0.912,1.085,0.462,1.136,0.373
4,regnety_016.tv2_in1k-c3p1b16e20f14,neural_foraminal,0.779,6.050,0.908,0.902,0.397,0.667,0.299


In [90]:
nf.sort_values("nfm_a", ascending=False).head(3)

,model,nf,nf_acc,nfn_cw,nfn_a,nfm_cw,nfm_a,nfs_cw,nfs_a
21,efficientnet_b4-DB-c3p1b16e20f14,neural_foraminal,0.723,10.449,0.765,0.711,0.650,0.424,0.373
1,densenet121-c3p1b16e20f14,neural_foraminal,0.778,8.321,0.851,0.900,0.617,1.357,0.284
16,densenet201-DB-c3p1b16e20f14,neural_foraminal,0.633,6.008,0.682,0.416,0.596,0.000,0.000


In [91]:
nf.sort_values("nfs_a", ascending=False).head(3)

,model,nf,nf_acc,nfn_cw,nfn_a,nfm_cw,nfm_a,nfs_cw,nfs_a
5,rexnet_150.nav_in1k-A-c9p1b16e20f14,neural_foraminal,0.179,2.268,0.119,0.261,0.289,0.057,0.672
24,efficientnet_b0-DB-c3p1b16e20f14,neural_foraminal,0.738,12.338,0.787,0.771,0.570,0.443,0.642
10,rexnet_150.nav_in1k-c3p1b16e20f14,neural_foraminal,0.706,8.440,0.792,0.755,0.401,0.246,0.612
